In [4]:

import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_upstage import UpstageEmbeddings # UpstageEmbeddings로 변경
from langchain_community.vectorstores import FAISS

# .env 파일에서 환경 변수 로드
load_dotenv()

# 1. 카페 메뉴 데이터 파일 로드 및 분할
print(">>> 1단계: 카페 메뉴 데이터를 로드하고 분할합니다...")
loader = TextLoader("../data/cafe_menu_data.txt", encoding="utf-8")
documents = loader.load()

text_splitter = CharacterTextSplitter(separator="\n\n---\n\n", chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
print(f"분할된 문서의 수: {len(docs)}")

# 2. Upstage 임베딩 모델 설정
print(">>> 2단계: Upstage 임베딩 모델을 설정합니다...")
embeddings = UpstageEmbeddings(model="solar-embedding-1-large") # OpenAI를 Upstage로 변경
vectorstore = FAISS.from_documents(docs, embeddings)

# 3. 벡터 DB 로컬에 저장
db_path = "./db/cafe_db"
if not os.path.exists(db_path):
    os.makedirs(db_path)

vectorstore.save_local(db_path)
print(f">>> 3단계: 벡터 DB를 '{db_path}' 경로에 성공적으로 저장했습니다.")

>>> 1단계: 카페 메뉴 데이터를 로드하고 분할합니다...
분할된 문서의 수: 1
>>> 2단계: Upstage 임베딩 모델을 설정합니다...
>>> 3단계: 벡터 DB를 './db/cafe_db' 경로에 성공적으로 저장했습니다.


In [7]:
# 파일명: 2_main_cafe_assistant.py
import os
from typing import List
from dotenv import load_dotenv
from langchain_core.tools import tool
from langchain_upstage import ChatUpstage, UpstageEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.messages import HumanMessage, ToolMessage

# .env 파일에서 환경 변수 로드
load_dotenv()

# --- 1. 3개의 도구 정의 ---

@tool
def tavily_search_func(query: str) -> str:
    """웹에서 최신 정보를 검색합니다. 최신 커피 트렌드, 특정 카페의 위치나 영업시간 등을 물어볼 때 유용합니다."""
    search = TavilySearchResults()
    return search.invoke(query)

@tool
def wiki_summary(query: str) -> str:
    """위키피디아에서 특정 주제에 대한 일반적인 지식이나 정보를 검색하고 요약합니다. 커피의 역사, 특정 음료의 유래나 제조법 등을 물어볼 때 사용하세요."""
    wikipedia = WikipediaAPIWrapper(lang="ko", load_all_available_meta=False, top_k_results=1)
    return wikipedia.run(query)

@tool
def db_search_cafe_func(query: str) -> List[str]:
    """로컬 카페 메뉴 데이터베이스에서 메뉴 정보를 검색합니다. 메뉴의 가격, 재료, 설명 등을 물어볼 때 사용해야 합니다."""
    # 벡터 DB 로드
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    vectorstore = FAISS.load_local("./db/cafe_db", embeddings, allow_dangerous_deserialization=True)
    
    # 유사도 검색
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
    result_docs = retriever.invoke(query)
    
    # Document 객체의 page_content만 추출하여 리스트로 반환
    return [doc.page_content for doc in result_docs]

# --- 2. LLM에 도구 바인딩 ---

# 도구 리스트 생성
tools = [tavily_search_func, wiki_summary, db_search_cafe_func]

# LLM 모델 설정 (Upstage Solar 모델 사용)
llm = ChatUpstage(model_name="solar-1-mini-chat", temperature=0)

# LLM에 도구 바인딩
llm_with_tools = llm.bind_tools(tools)

# --- 3. 간단한 도구 호출 체인 구현 ---

def run_cafe_assistant(query: str):
    """사용자 질문에 따라 적절한 도구를 호출하고 최종 답변을 생성하는 체인"""
    print(f"\n[사용자 질문]\n{query}")

    # 1. 첫 번째 LLM 호출: 질문을 분석하고 어떤 도구를 사용할지 결정
    message = HumanMessage(content=query)
    ai_msg = llm_with_tools.invoke([message])
    
    # 도구 호출이 없으면 바로 답변 반환
    if not ai_msg.tool_calls:
        print("\n[최종 답변]")
        return ai_msg.content

    print("\n[LLM의 도구 선택]")
    print(ai_msg.tool_calls)

    # 2. 도구 실행 및 결과 저장
    tool_messages = []
    for tool_call in ai_msg.tool_calls:
        # 호출할 도구 이름으로 실제 도구 함수를 찾음
        selected_tool = {
            "tavily_search_func": tavily_search_func,
            "wiki_summary": wiki_summary,
            "db_search_cafe_func": db_search_cafe_func,
        }.get(tool_call["name"])
        
        # 도구 실행
        if selected_tool:
            tool_output = selected_tool.invoke(tool_call["args"])
            print(f"\n[{tool_call['name']} 실행 결과]")
            print(tool_output)
            
            # ToolMessage 생성
            tool_messages.append(
                ToolMessage(content=str(tool_output), tool_call_id=tool_call["id"])
            )

    # 3. 두 번째 LLM 호출: 도구 실행 결과를 바탕으로 최종 답변 생성
    final_response = llm_with_tools.invoke([message] + [ai_msg] + tool_messages)
    
    print("\n[최종 답변]")
    return final_response.content


# --- 4. 테스트 질문 처리 ---
test_query = "아메리카노의 가격과 특징은 무엇인가요?"
final_answer = run_cafe_assistant(test_query)
print(final_answer)

# 추가 테스트
# test_query_2 = "카푸치노의 역사에 대해 알려줘."
# final_answer_2 = run_cafe_assistant(test_query_2)
# print(final_answer_2)


[사용자 질문]
아메리카노의 가격과 특징은 무엇인가요?

[LLM의 도구 선택]
[{'name': 'db_search_cafe_func', 'args': {'query': '아메리카노'}, 'id': '4ed96c3d-a194-43f5-9411-63cb01ec2b0f', 'type': 'tool_call'}]

[db_search_cafe_func 실행 결과]
['1. 아메리카노\n   • 가격: ₩4,500\n   • 주요 원료: 에스프레소, 뜨거운 물\n   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.\n\n2. 카페라떼\n   • 가격: ₩5,500\n   • 주요 원료: 에스프레소, 스팀 밀크\n   • 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한 질감과 부드러운 맛이 특징이며, 다양한 시럽과 토핑 추가가 가능합니다. 라떼 아트로 시각적 즐거움도 제공합니다.\n\n3. 카푸치노\n   • 가격: ₩5,000\n   • 주요 원료: 에스프레소, 스팀 밀크, 우유 거품\n   • 설명: 에스프레소, 스팀 밀크, 우유 거품이 1:1:1 비율로 구성된 이탈리아 전통 커피입니다. 진한 커피 맛과 부드러운 우유 거품의 조화가 일품이며, 계피 파우더를 뿌려 제공합니다.\n\n4. 바닐라 라떼\n   • 가격: ₩6,000\n   • 주요 원료: 에스프레소, 스팀 밀크, 바닐라 시럽\n   • 설명: 카페라떼에 달콤한 바닐라 시럽을 더한 인기 메뉴입니다. 바닐라의 달콤함과 커피의 쌉싸름함이 조화롭게 어우러지며, 휘핑크림 토핑으로 더욱 풍성한 맛을 즐길 수 있습니다.\n\n5. 카라멜 마키아토\n   • 가격: ₩6,500\n   • 주요 원료: 에스프레소, 스팀 밀크, 카라멜 시럽, 휘핑크림\n   • 설명: 스팀 밀크 위에 에스프레소를 부어 만든 후 카라멜 시럽과 휘핑크림